In [1]:
from p2pfl.node import Node
from p2pfl.learning.learner import MyNodeLearning
from p2pfl.agregator import FedAvg    
from collections import OrderedDict
import torch
import time


def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(host="localhost")
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía

        
test_convergence((2,2))

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmp3zph9zaw
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/vm/p9lv42ms7ml71cjvsj4bnbnm0000gn/T/tmp3zph9zaw/_remote_module_non_sriptable.py
INFO:root:Nodo a la escucha en localhost 50140
INFO:root:Nodo a la escucha en localhost 50141
INFO:root:Conexión aceptada con ('localhost', 50141)
INFO:root:Broadcasting start learning...
INFO:root:Training...


GPU available: False, used: False


INFO:root:Training...


TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params si

Train: 54000 Val:6000 Test:10000Train: 54000 Val:6000 Test:10000



...

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)


Train: 54000 Val:6000 Test:10000
Train: 54000 Val:6000 Test:10000


Sanity Checking: 0it [00:00, ?it/s]

Sanity Checking: 0it [00:00, ?it/s]

INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/2)
INFO:root:Agregating models.
INFO:root:Round 2 of 2 finished. (('localhost', 50140))
INFO:root:Finish!!.
INFO:root:Model added (2/2)
INFO:root:Agregating models.
INFO:root:Round 2 of 2 finished. (('localhost', 50141))
INFO:root:Finish!!.
INFO:root:Bajando el nodo, dejando de escuchar en localhost 50140
DEBUG:root:Closed connection: ('localhost', 50140)
DEBUG:root:Closed connection: ('localhost', 50141)
INFO:root:Bajando el nodo, dejando de escuchar en localhost 50141


In [3]:
from p2pfl.const import HEARTBEAT_FREC, TIEMOUT
from p2pfl.node import Node
import time

n1 = Node("localhost")
n2 = Node("localhost")
n1.start()
n2.start()

def test_interrupt_train2(two_nodes):
    n1, n2 = two_nodes
    n1.connect_to(n2.host,n2.port)

    time.sleep(1) #Esperar por la asincronía

    n1.set_start_learning(99999,99999)

    time.sleep(1) #Esperar por la asincronía

    n2.set_stop_learning()
    
    time.sleep(1) #Esperar por la asincronía
    
    print(n1.round)
    print(n2.round)
    
    assert n1.round is None
    
test_interrupt_train2((n1,n2))

INFO:root:Nodo a la escucha en localhost 65058
INFO:root:Nodo a la escucha en localhost 65059
INFO:root:Conexión aceptada con ('localhost', 65058)
INFO:root:Broadcasting start learning...
INFO:root:Training...


GPU available: False, used: False


INFO:root:Training...


TPU available: False, using: 0 TPU cores
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
HPU available: False, using: 0 HPUs

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params size (MB)

  | Name     | Type     | Params
--------------------------------------
0 | accuracy | Accuracy | 0     
1 | l1       | Linear   | 100 K 
2 | l2       | Linear   | 33.0 K
3 | l3       | Linear   | 2.6 K 
--------------------------------------
136 K     Trainable params
0         Non-trainable params
136 K     Total params
0.544     Total estimated model params si

Train: 54000 Val:6000 Test:10000Train: 54000 Val:6000 Test:10000



Sanity Checking: 0it [00:00, ?it/s]

Sanity Checking: 0it [00:00, ?it/s]

INFO:root:Stopping learning
INFO:root:Stopping learning
None
None


In [1]:
len(n1.neightboors)

NameError: name 'n1' is not defined